In [1]:
import pandas as pd

In [2]:
storage_path = "../../data/processed/historical_kba_data.parquet"
df = pd.read_parquet(storage_path, engine='fastparquet')
df["ts_key_size"] = df.groupby('ts_key')['ts_key'].transform('size')

# FIlter ts_keys with at least 12 entries
df = df[df['ts_key_size'] >= 12].copy()

In [3]:
df.head()

,OEM,Model,drive_type,Value,Date,ts_key,ts_key_size
1745,ALFA ROMEO,GIULIA,All_Wheel_Drive,79.0,2022-10-31,ALFA ROMEO_GIULIA_All_Wheel_Drive,94
2094,ALFA ROMEO,GIULIA,Convertibles,0.0,2022-10-31,ALFA ROMEO_GIULIA_Convertibles,94
698,ALFA ROMEO,GIULIA,Diesel,9.0,2022-10-31,ALFA ROMEO_GIULIA_Diesel,94
1047,ALFA ROMEO,GIULIA,Electric_BEV,0.0,2022-10-31,ALFA ROMEO_GIULIA_Electric_BEV,94
1396,ALFA ROMEO,GIULIA,Hybrid,0.0,2022-10-31,ALFA ROMEO_GIULIA_Hybrid,94


In [4]:
columns = ['Date','ts_key', 'Value']

df = df[columns].copy()

In [5]:
df.head()

,Date,ts_key,Value
1745,2022-10-31,ALFA ROMEO_GIULIA_All_Wheel_Drive,79.0
2094,2022-10-31,ALFA ROMEO_GIULIA_Convertibles,0.0
698,2022-10-31,ALFA ROMEO_GIULIA_Diesel,9.0
1047,2022-10-31,ALFA ROMEO_GIULIA_Electric_BEV,0.0
1396,2022-10-31,ALFA ROMEO_GIULIA_Hybrid,0.0


In [6]:
df.ts_key.nunique()

3745

In [14]:
# Aggregate duplicates by summing values
pivot_df = df.groupby(['Date', 'ts_key'])['Value'].sum().unstack(fill_value=0)

: 

: 

# LSTM Time Series Forecasting

## Plan:
1. **Data Analysis**: Examine time series lengths and prepare for uniform processing
2. **Dataset Creation**: Create sliding window sequences (6 months input → 1 month output)
3. **Model Architecture**: Build LSTM model in PyTorch
4. **Training Setup**: Configure train-test split, data loaders, and MPS device
5. **Training**: Train single model across all time series
6. **Evaluation**: Assess 1-month ahead forecast performance

## Key Parameters:
- Time series: 3,745 unique keys
- Sequence length: 6 months (lookback)
- Forecast horizon: 1 month ahead
- Device: MPS (Apple Silicon GPU)


## Step 1: Data Analysis and Preparation

In [9]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Check MPS availability
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"✓ MPS device is available")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✓ CUDA device is available")
else:
    device = torch.device("cpu")
    print(f"Using CPU")
    
print(f"Device: {device}")


✓ MPS device is available
Device: mps


In [ ]:
class TimeSeriesDataset(Dataset):
    """
    Dataset for time series with sliding window approach.
    Each sample: [y(t-6), y(t-5), ..., y(t-1)] -> y(t)
    """
    def __init__(self, df, seq_length=6, train=True, train_ratio=0.8):
        self.seq_length = seq_length
        self.X = []
        self.y = []
        self.ts_keys = []
        
        # Group by time series key
        grouped = df.sort_values('Date').groupby('ts_key')
        
        for ts_key, group in grouped:
            values = group['Value'].values
            
            # Skip if not enough data
            if len(values) < seq_length + 1:
                continue
            
            # Create sliding windows
            for i in range(len(values) - seq_length):
                self.X.append(values[i:i+seq_length])
                self.y.append(values[i+seq_length])
                self.ts_keys.append(ts_key)
        
        self.X = np.array(self.X, dtype=np.float32)
        self.y = np.array(self.y, dtype=np.float32)
        
        # Train-test split
        n_samples = len(self.X)
        train_size = int(n_samples * train_ratio)
        
        if train:
            self.X = self.X[:train_size]
            self.y = self.y[:train_size]
            self.ts_keys = self.ts_keys[:train_size]
        else:
            self.X = self.X[train_size:]
            self.y = self.y[train_size:]
            self.ts_keys = self.ts_keys[train_size:]
        
        # Standardize
        self.scaler_X = StandardScaler()
        self.scaler_y = StandardScaler()
        
        # Fit scaler on training data
        if train:
            self.X = self.scaler_X.fit_transform(self.X)
            self.y = self.scaler_y.fit_transform(self.y.reshape(-1, 1)).flatten()
        else:
            # For test set, we need the scaler from training
            # This will be handled separately
            pass
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.X[idx]).unsqueeze(-1), torch.FloatTensor([self.y[idx]])


TimeSeriesDataset class defined


In [10]:
class LSTMForecaster(nn.Module):
    """
    LSTM model for univariate time series forecasting.
    Architecture: LSTM -> Dropout -> LSTM -> Dropout -> Fully Connected
    """
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, dropout=0.2):
        super(LSTMForecaster, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        # x shape: (batch_size, seq_length, input_size)
        
        # LSTM forward pass
        lstm_out, (h_n, c_n) = self.lstm(x)
        
        # Take the output from the last time step
        last_output = lstm_out[:, -1, :]
        
        # Apply dropout
        out = self.dropout(last_output)
        
        # Fully connected layer
        out = self.fc(out)
        
        return out

# Initialize model
model = LSTMForecaster(
    input_size=1,
    hidden_size=64,
    num_layers=2,
    dropout=0.2
).to(device)

print(f"Model architecture:\n{model}")
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")


NameError: name 'nn' is not defined

In [ ]:
# Create datasets
SEQ_LENGTH = 6  # 6 months lookback
BATCH_SIZE = 256
TRAIN_RATIO = 0.8

print("Creating training dataset...")
train_dataset = TimeSeriesDataset(df, seq_length=SEQ_LENGTH, train=True, train_ratio=TRAIN_RATIO)
print(f"Training samples: {len(train_dataset):,}")

# Save scalers for test set
scaler_X = train_dataset.scaler_X
scaler_y = train_dataset.scaler_y

print("\nCreating test dataset...")
test_dataset = TimeSeriesDataset(df, seq_length=SEQ_LENGTH, train=False, train_ratio=TRAIN_RATIO)

# Apply training scalers to test set
test_dataset.X = scaler_X.transform(test_dataset.X)
test_dataset.y = scaler_y.transform(test_dataset.y.reshape(-1, 1)).flatten()
test_dataset.scaler_X = scaler_X
test_dataset.scaler_y = scaler_y

print(f"Test samples: {len(test_dataset):,}")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

print(f"\nBatches per epoch: {len(train_loader)}")
print(f"Test batches: {len(test_loader)}")


In [ ]:
from tqdm.notebook import tqdm
import time

def train_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        predictions = model(X_batch)
        loss = criterion(predictions, y_batch)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
    
    return total_loss / len(loader)

def evaluate(model, loader, criterion, device):
    """Evaluate model"""
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            predictions = model(X_batch)
            loss = criterion(predictions, y_batch)
            total_loss += loss.item()
    
    return total_loss / len(loader)

print("Training functions defined")


In [ ]:
# Training loop
train_losses = []
test_losses = []
best_test_loss = float('inf')

print(f"Starting training on {device}...")
print("=" * 60)

start_time = time.time()

for epoch in range(EPOCHS):
    # Train
    train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    
    # Evaluate
    test_loss = evaluate(model, test_loader, criterion, device)
    test_losses.append(test_loss)
    
    # Learning rate scheduling
    scheduler.step(test_loss)
    
    # Save best model
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), '../../models/best_lstm_model.pth')
    
    # Print progress
    if (epoch + 1) % 5 == 0 or epoch == 0:
        elapsed = time.time() - start_time
        print(f"Epoch [{epoch+1:3d}/{EPOCHS}] | "
              f"Train Loss: {train_loss:.6f} | "
              f"Test Loss: {test_loss:.6f} | "
              f"Time: {elapsed:.1f}s")

total_time = time.time() - start_time
print("=" * 60)
print(f"Training completed in {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"Best test loss: {best_test_loss:.6f}")


In [ ]:
# Calculate metrics on test set
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load best model
model.load_state_dict(torch.load('../../models/best_lstm_model.pth'))
model.eval()

# Get predictions
all_predictions = []
all_actuals = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        predictions = model(X_batch)
        all_predictions.extend(predictions.cpu().numpy())
        all_actuals.extend(y_batch.numpy())

# Convert to numpy arrays
all_predictions = np.array(all_predictions).flatten()
all_actuals = np.array(all_actuals).flatten()

# Inverse transform to original scale
all_predictions_original = scaler_y.inverse_transform(all_predictions.reshape(-1, 1)).flatten()
all_actuals_original = scaler_y.inverse_transform(all_actuals.reshape(-1, 1)).flatten()

# Calculate metrics
mae = mean_absolute_error(all_actuals_original, all_predictions_original)
rmse = np.sqrt(mean_squared_error(all_actuals_original, all_predictions_original))
mape = np.mean(np.abs((all_actuals_original - all_predictions_original) / (all_actuals_original + 1e-8))) * 100
r2 = r2_score(all_actuals_original, all_predictions_original)

print("=" * 60)
print("MODEL PERFORMANCE METRICS (Original Scale)")
print("=" * 60)
print(f"Mean Absolute Error (MAE):        {mae:.2f}")
print(f"Root Mean Squared Error (RMSE):   {rmse:.2f}")
print(f"Mean Absolute Percentage Error:   {mape:.2f}%")
print(f"R² Score:                         {r2:.4f}")
print("=" * 60)


## Summary

**Model Configuration:**
- Architecture: 2-layer LSTM with 64 hidden units
- Input: 6-month sliding window
- Output: 1-month ahead forecast
- Training on MPS (Apple Silicon GPU)
- Single model trained across all 3,745 time series

**Key Features:**
- Standardized input/output using StandardScaler
- Dropout (0.2) for regularization
- Gradient clipping to prevent exploding gradients
- Learning rate scheduling with ReduceLROnPlateau
- 80/20 train-test split

**Next Steps:**
1. Experiment with different sequence lengths
2. Try multivariate approach with exogenous variables
3. Compare with other architectures (GRU, Transformer)
4. Implement per-time-series evaluation
5. Analyze performance by OEM/powertrain type


In [ ]:
# Visualization: Predictions vs Actuals
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Scatter plot
axes[0].scatter(all_actuals_original, all_predictions_original, alpha=0.3, s=10)
axes[0].plot([all_actuals_original.min(), all_actuals_original.max()], 
             [all_actuals_original.min(), all_actuals_original.max()], 
             'r--', linewidth=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual Values', fontsize=12)
axes[0].set_ylabel('Predicted Values', fontsize=12)
axes[0].set_title('Predictions vs Actuals (All Time Series)', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)

# Residuals
residuals = all_actuals_original - all_predictions_original
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Residuals (Actual - Predicted)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Residual Distribution', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Residual statistics:")
print(f"  Mean: {residuals.mean():.2f}")
print(f"  Std: {residuals.std():.2f}")
print(f"  Min: {residuals.min():.2f}")
print(f"  Max: {residuals.max():.2f}")


In [ ]:
import matplotlib.pyplot as plt

# Plot training history
fig, ax = plt.subplots(1, 1, figsize=(12, 5))

ax.plot(train_losses, label='Training Loss', linewidth=2)
ax.plot(test_losses, label='Test Loss', linewidth=2)
ax.set_xlabel('Epoch', fontsize=12)
ax.set_ylabel('Loss (MSE)', fontsize=12)
ax.set_title('LSTM Training History - All Time Series', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Final Training Loss: {train_losses[-1]:.6f}")
print(f"Final Test Loss: {test_losses[-1]:.6f}")


## Step 6: Evaluation and Visualization

In [ ]:
# Training configuration
EPOCHS = 50
LEARNING_RATE = 0.001

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

print(f"Training configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Optimizer: Adam")
print(f"  Loss function: MSE")
print(f"  Device: {device}")


## Step 5: Training Setup and Loop

## Step 4: Prepare Data Loaders

## Step 3: Define LSTM Model Architecture

## Step 2: Create Sliding Window Dataset

In [ ]:
# Analyze time series lengths
ts_lengths = df.groupby('ts_key').size()
print(f"Total time series: {df.ts_key.nunique()}")
print(f"\nTime series length statistics:")
print(ts_lengths.describe())
print(f"\nMinimum length: {ts_lengths.min()}")
print(f"Maximum length: {ts_lengths.max()}")

# Check how many series have at least 7 observations (needed for 6-month lookback + 1 target)
min_required = 7
valid_series = (ts_lengths >= min_required).sum()
print(f"\nTime series with >= {min_required} observations: {valid_series} / {len(ts_lengths)}")
